In [1]:
!pwd

/Users/nakatani/labo/2021_root/2021_Dec/refine_diff/pynb


In [2]:
import os
import cv2
import re
import numpy as np

In [5]:
IMG = '../../noise-removed'
DIFF = '../diff'

In [6]:
fnames = sorted(os.listdir(os.path.join(IMG)))

## diff取り出しの実験

boldじゃやないpile-upとの差分

In [10]:
prev_root = ''
kernel = np.ones((5,5),np.uint8)
for fname in fnames:
    root = re.findall('(.+)_', fname)[0]
    img = cv2.imread(os.path.join(IMG, fname))
    if prev_root != root:
        base_img = img
        prev_root = root
        continue
    dilate = cv2.dilate(base_img,kernel,iterations = 1)
    bold_not = cv2.bitwise_not(dilate)[:,:,0].squeeze()
    diff = cv2.bitwise_and(img, img, mask = bold_not)
    base_img = base_img + diff
    cv2.imwrite(os.path.join('../diff_pile-up_itr1', fname), diff)
    
#     prev_img = img
    prev_root = root

ちょいずれによる差分は画素値小さいのでthereshで除去を試みる，thresh前とどれくらい違うのかを確認するためbitwiseORで色を変えて重ねる

In [18]:
prev_root = ''
kernel = np.ones((5,5),np.uint8)
for fname in fnames:
    root = re.findall('(.+)_', fname)[0]
    img = cv2.imread(os.path.join(IMG, fname))
    if prev_root != root:
        base_img = img
        prev_root = root
        continue
    dilate = cv2.dilate(base_img,kernel,iterations = 1)
    bold_not = cv2.bitwise_not(dilate)[:,:,0].squeeze()
    diff = cv2.bitwise_and(img, img, mask = bold_not)
    base_img = base_img + diff
    ret,thresh = cv2.threshold(diff,100,255,0)
    cv2.imwrite(os.path.join('../diff_pile-up_itr1_thresh', fname), thresh)
    ret, diff_ = cv2.threshold(diff,1,255,0)
    diff_[:,:,:2] = 0
    dst = cv2.bitwise_or(thresh, diff_)
    cv2.imwrite(os.path.join('../check_thresh/', fname), dst)
#     prev_img = img
    prev_root = root

In [19]:
DST = '../diff_er2dl'

In [ ]:
kernel = np.ones((3,3),np.uint8)
for fname in fnames:
    thre = cv2.imread(os.path.join(THRE, fname))
    erosion = cv2.erode(thre,kernel,iterations = 1)
    erosion_ = cv2.dilate(erosion,kernel,iterations = 1)
#     cv2.imwrite(os.path.join(DST, fname), erosion_)

In [20]:
import re

In [31]:
for fname in fnames:
    date = re.findall('_(\d+)', fname)[0]
    if date == '211122':
        base = cv2.imread('../../noise-removed/' + fname)
    else:
        diff = cv2.imread(os.path.join(DST, fname))
#         diff_red = diff + 0
#         diff_red[:,:,:2] = 1
#         base_or_diff = cv2.bitwise_or(base, diff_red)
#         base_or_diff_ = cv2.dilate(base_or_diff, kernel)
#         base_or_diff_ = cv2.erode(base_or_diff_, kernel)
#         cv2.imwrite('../check_connect/' + fname, base_or_diff_)
        base_or_diff = cv2.bitwise_or(base, diff)
        base_or_diff_ = cv2.dilate(base_or_diff, kernel)
        base_or_diff_ = cv2.erode(base_or_diff_, kernel)
        base_not = cv2.bitwise_not(base)
        diff_ = cv2.bitwise_and(base_or_diff_, base_not)
        base = base_or_diff_
        cv2.imwrite('../diff_connect/' + fname, diff_)
#         base = base
#         base_or_diff_ = cv2.erode(base_or_diff_, kernel)